In [1]:
from ipywidgets import GridspecLayout, Layout, Label, Box, HBox, VBox, HTML, widgets, Button
from IPython.display import display

from risotto.artifacts import load_papers_artifact, load_papers_topics_artifacts, load_topics_artifacts

In [2]:
papers_artifact = load_papers_artifact()
papers_topics_artifact = load_papers_topics_artifacts()
topics_artifact = load_topics_artifacts()

# Mock topics and subtopics data
topics = ["A", "B", "C", "D"]
subtopics = {
    "Todos": [],
    "A": ["A1", "A2"],
    "B": ["B1", "B2"],
    "C": ["C1", "C2"],
    "D": ["D1", "D2"],
}

# Instantiate filter & search widgets
dropdown_layout = Layout()
topics_dropdown = widgets.Dropdown(
    options=["Todos"] + topics,
    description="Tópico:",
    layout=dropdown_layout,
)
subtopics_dropdown = widgets.Dropdown(
    options=["Todos"] + subtopics[topics_dropdown.value],
    description="Subtópico:",
    layout=dropdown_layout,
)
search_textbox = widgets.Text(
    description="Búsqueda:",
)
search_button = widgets.Button(
    description="Filtrar",
)
dropdowns_box = HBox([
    topics_dropdown,
    subtopics_dropdown,
])
search_box = HBox([
    search_textbox,
    search_button,
])
filter_and_search_box = VBox([
    dropdowns_box,
    search_box,
])

# Event handlers
def handle_topic_change(event):
    if event["type"] == "change" and event["name"] == "value":
        subtopics_dropdown.options = ["Todos"] + subtopics[event["new"]]

def handle_subtopic_change(event):
    if event["type"] == "change" and event["name"] == "value":
        pass

topics_dropdown.observe(handle_topic_change)
subtopics_dropdown.observe(handle_subtopic_change)

filter_and_search_box

In [3]:
class PaperView:
    def __init__(self, row):
        self._row = row
        
    def to_widget(self):
        grid = GridspecLayout(n_rows=7, n_columns=2, justify_content='space-around', align_items='flex-start')

        grid[0,0] = Label('Title:', layout=Layout(height='auto', width='20%'))
        grid[0,1] = Label(self._row.title)

        grid[1,0] = Label('Authors:')
        grid[1,1] = Label(self._row.authors)

        grid[2,0] = Label('Date:', layout=Layout(height='auto', width='20%'))
        grid[2,1] = Label(self._row.publish_time)
        grid[3,0] = Label('Relevance:', layout=Layout(height='auto', width='20%'))
        grid[3,1] = Label(str(self._row.pagerank))
        grid[4,0] = Label('URL:',layout=Layout(height='auto', width='20%'))
        grid[4,1] = HTML(f'<a href="{self._row.url}">{self._row.url}</a>')
        grid[5,0] = Label('DOI:',layout=Layout(height='auto', width='20%'))
        grid[5,1] = Label(f'{self._row.doi}')
        grid[6,0] = Label('Abstract:', layout=Layout(height='auto', width='20%'))
        grid[6,1] = Label(str(self._row.abstract))

        return grid

    
class PaperSetView:
    def __init__(self, papers, page, items_per_page, parent):
        self._papers = papers
        self._page = page
        self._items_per_page = items_per_page
        self._parent = parent
        self._papers_views = self.build_papers_views(papers, page, items_per_page)
        
        self._parent.children = [self.to_widget()]
    
    def _decrement_page(self, _):
        self._page += -1
        self._papers_views = self.build_papers_views(self._papers, self._page, self._items_per_page)
        self._parent.children = [self.to_widget()]
    
    def _increment_page(self, _):
        self._page += 1
        self._papers_views = self.build_papers_views(self._papers, self._page, self._items_per_page)
        self._parent.children = [self.to_widget()]
    
    @staticmethod
    def build_papers_views(papers, page, items_per_page):
        start_idx, end_idx = page * items_per_page, (page + 1) * items_per_page
        page_subset = papers.iloc[start_idx:end_idx]
        return [PaperView(row) for _, row in page_subset.iterrows()]
    
    @staticmethod
    def get_nav_widgets(page, items_per_page, num_items, decrement_handler, increment_handler):
        last_page = num_items // items_per_page
        widgets = []
        if page > 0:
            button = Button(description="Página anterior")
            button.on_click(decrement_handler)
            widgets.append(button)
        widgets.append(HTML(f"Página {page + 1} de {last_page + 1}. Mostrando {items_per_page} elementos por página."))
        if page < last_page:
            button = Button(description="Página siguiente")
            button.on_click(increment_handler)
            widgets.append(button)
        return HBox(widgets)
    
    def to_widget(self):
        papers_widgets = [view.to_widget() for view in self._papers_views]
        nav_widgets = self.get_nav_widgets(
            page=self._page,
            items_per_page=self._items_per_page,
            num_items=len(self._papers),
            decrement_handler=self._decrement_page,
            increment_handler=self._increment_page
        )
        box = VBox(papers_widgets + [nav_widgets])
        return box

box = Box()
paper_set_view = PaperSetView(papers_artifact, page=0, items_per_page=5, parent=box)

box

Box(children=(VBox(children=(GridspecLayout(children=(Label(value='Title:', layout=Layout(grid_area='widget001…

In [4]:
"""
TODO:
- Integrar y encapsular componentes de filtro y búsqueda
- Integrar Whoosh para la búsqueda
- Hacer encabezado
"""

'\nTODO:\n- Integrar y encapsular componentes de filtro y búsqueda\n- Integrar Whoosh para la búsqueda\n- Hacer encabezado\n'